## RedisやKeyDBによるngramの集計コード
適宜分割して複数並列で実行することをお勧めします

In [1]:
import redis
pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
client = redis.StrictRedis(connection_pool=pool)

In [4]:
pipe = client.pipeline()

In [ ]:
import os
import tarfile
import glob
import gzip
import json
from tqdm import tqdm
import random
import pandas as pd
random.seed(777)
#1_ngrammakerで作成した資料ID毎のngram集計情報のjson
INPUTDIR="/data/ngramviewer/ngramdata/"
cnt=0
inputgzfilelist=glob.glob(os.path.join(INPUTDIR,"*.json.gz"))
random.shuffle(inputgzfilelist)
print(len(inputgzfilelist))
tmpval=0
allcnt=0
for inputgzpath in tqdm(inputgzfilelist):
    pid=os.path.basename(inputgzpath).split(".")[0]
    dfrow=dfmini2[dfmini2["PID"]==pid]
    yeararray=dfrow["出版年(W3CDTF)"].values
    if len(yeararray)>0:
        cnt+=1
        with gzip.open(inputgzpath, mode='rt') as fp:
            jsonobj=json.load(fp)
            for k,v in jsonobj.items():
                try:
                    k=str(k)
                    k=k.replace(" ","")
                    pipe.hincrby(k,yeararray[0],v)
                except:
                    pass
        if cnt%200==0:#200冊溜まったらバルク投入する
            pipe.execute()
            pipe = client.pipeline()
pipe.execute()